In [2]:
import numpy as np
import pandas as pd
import os, random, shutil, sys, time
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import sys

#mac
util_path = "/Users/kimd999/research/projects/Katrina/srpAnalytics/code/latest/util"

#constance
'''args = sys.argv[0:]
py_file = args[0]

code_location = os.path.dirname(os.path.abspath(py_file))
index_of_latest = code_location.index('latest')
util_path = os.path.join(code_location[:index_of_latest], "latest", "util")
'''

sys.path.insert(0, util_path)

# These two lines will automatically reload updated modules of mine
# (therefore, I don't need to restart kernel after editing importing modules)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/research/projects/Katrina/srpAnalytics/code/latest/2_bmd/2_analysis/Bree/CTG


In [48]:
# keep record - worked for CTG
complete_file_path = '/Users/kimd999/research/projects/Katrina/per_each_data/Bree/run_2021_Nov/run_May_data_in_Nov/input/BHT_May.csv'
#complete_file_path = '/Users/kimd999/research/projects/Katrina/per_each_data/Bree/run_2021_Nov/run_May_data_in_Nov/input/tonalide_min_value_from_10_uM_May.csv'
df_test_dose_response = pd.read_csv(complete_file_path, header = 0)
pd.set_option('display.max_columns', None)
display(df_test_dose_response.head())
#display(np.unique(morpho_data.well))

,dose,num_affected,total_num
0,0,1.519964,100
1,10,0.000000,100
2,25,6.893015,100
3,50,26.519651,100
4,75,36.724537,100


In [25]:
import generate_dose_response as gdr
import BMD_BMDL_estimation as bmdest
import Plot_Save as ps

In [49]:
# Fit dose response models
start_time = time.time()

df_test_dose_response = df_test_dose_response.sort_values(by=['dose'])
print (f"df_test_dose_response:\n{df_test_dose_response}")

model_predictions = bmdest.analyze_dose_response_data(df_test_dose_response)
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("BMD calculation is done, it took:"+str(time_took)) 

df_test_dose_response:
   dose  num_affected  total_num
0     0      1.519964        100
1    10      0.000000        100
2    25      6.893015        100
3    50     26.519651        100
4    75     36.724537        100
5   100     62.295397        100
6   200     94.624672        100
BMD calculation is done, it took:0.1 seconds


In [50]:
print (model_predictions)

          Model  Chi-squared     p-val         AIC     BMD10     BMDL10  \
0  log_logistic     7.458424  0.113557  502.321889  35.91109  30.713144   

       BMD50                                   Scaled Residuals  \
0  82.034986  [-0.5616445303213036, 1.1633563456567928, -0.8...   

                                    Optimized Params Conv Flag BMDL Conv  
0  [0.00969608094888539, -11.721938240772209, 2.6...         0         0  


In [51]:
# Select best model
selected_model_params = bmdest.select_model(model_predictions)
print (selected_model_params)

{'model': 0    log_logistic
Name: Model, dtype: object, 'no_unique_model_found_flag': 0, 'model_select_flag': 0, 'pval_thresh_flag': 0}


In [52]:
chemical_id = "tonalide_min_value_from_10_uM_by_log_logistic"
end_point = "CTG"
bmd_feasibility_flag = 2

# Check if unique model is found
unique_model_flag = selected_model_params['no_unique_model_found_flag']

if(unique_model_flag == 0):
    # Generate report
    #print(df_test_dose_response.dose[-1:])

    ps.save_results_good_data_unique_model(df_test_dose_response, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)
else:
    bmd_analysis_flag = selected_model_params['model_select_flag']
    if(bmd_analysis_flag == 1):
        ps.save_results_poor_data_or_no_convergence(df_test_dose_response, bmd_feasibility_flag, str(chemical_id), end_point, selected_model_params)
    else:
        ps.save_results_good_data_nounique_model(df_test_dose_response, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)